###Atualização do pip

In [1]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Instalação do Apache Beam

In [2]:
pip install apache-beam[interactive]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install apache_beam[gcp]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Google Could Storage File System

In [4]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Importação das Bibliotecas

In [5]:
import apache_beam as beam

from google.cloud import storage
import os

### Inserção do token da conta de serviço

In [6]:
serviceAccount = '/content/projeto-katia146-5bb4add28a7f.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

###Pipeline Derivados de Petróleo

In [7]:
# Tabela retornando o produto, estado e valor total de volume

ufs = [
 'AC',
 'AL',
 'AM',
 'AP',
 'BA',
 'CE',
 'DF',
 'ES',
 'GO',
 'MA',
 'MG',
 'MS',
 'MT',
 'PA',
 'PB',
 'PE',
 'PI',
 'PR',
 'RJ',
 'RN',
 'RO',
 'RR',
 'RS',
 'SC',
 'SE',
 'SP',
 'TO'
 ]
 
produtos = [
          'Etanol Hidratado', 
          'GLP', 
          'Gasolina C', 
          'Gasolina de Aviação', 
          'Querosene Iluminante', 
          'Querosene de Aviação', 
          'Óleo Combustível', 
          'Óleo Diesel', 
]


for j in range(len(ufs)):
  for i in range(len(produtos)):
    pij = beam.Pipeline()
    pipe1 = (
            pij
            |f'Extrair do CSV{i}{j}'>> beam.io.ReadFromText('/content/dataset_final_derivados.csv',skip_header_lines = 0)
            |f'Separador do CSV{i}{j}'>> beam.Map(lambda record: record.split(','))
            |f'Filtrar derivado{i}{j}'>> beam.Filter(lambda record: (record[2] == produtos[i] and (record[1])==ufs[j]))
            |f'Agregar as Colunas{i}{j}'>> beam.Map(lambda record: (record[1],round(float(record[4]))))
            |f'Construir uma nova tabela{i}{j}'>> beam.CombinePerKey(sum)
            |f'1Construir uma nova tabela de derivados de petróleo{i}{j}'>> beam.Distinct()
            
        )

    pipe2 = (
            pij
            |f'{i}{j}Extrair do CSV'>> beam.io.ReadFromText('/content/dataset_final_derivados.csv',skip_header_lines = 0)
            |f'{i}{j}Separador do CSV'>> beam.Map(lambda record: record.split(','))
            |f'{i}{j}Filtrar derivados petrol'>> beam.Filter(lambda record: (record[2] == produtos[i] and (record[1])==ufs[j]))
            |f'{i}{j}Agregar as Colunas'>> beam.Map(lambda record: (record[1],record[2]))
            |f'{i}{j}Construir uma nova tabela de derivados petrol'>> beam.Distinct()
          
        )

    tabela = (
        {'Produto':pipe2, 'Total do volume':pipe1 }
        |f'{i}{j}Agrupar as pernas' >> beam.CoGroupByKey()
        |f'{i}{j}Resultado'>> beam.Map(print)
      
    )


    pij.run()

('AC', {'Produto': ['Etanol Hidratado'], 'Total do volume': [130249]})
('AC', {'Produto': ['GLP'], 'Total do volume': [667]})
('AC', {'Produto': ['Gasolina C'], 'Total do volume': [2144]})
('AC', {'Produto': ['Gasolina de Aviação'], 'Total do volume': [3715]})
('AC', {'Produto': ['Querosene de Aviação'], 'Total do volume': [100879]})
('AC', {'Produto': ['Óleo Combustível'], 'Total do volume': [1470]})
('AC', {'Produto': ['Óleo Diesel'], 'Total do volume': [3726]})
('AL', {'Produto': ['Etanol Hidratado'], 'Total do volume': [1109]})
('AL', {'Produto': ['GLP'], 'Total do volume': [3555]})
('AL', {'Produto': ['Gasolina C'], 'Total do volume': [6692]})
('AL', {'Produto': ['Gasolina de Aviação'], 'Total do volume': [1247]})
('AL', {'Produto': ['Querosene de Aviação'], 'Total do volume': [5545]})
('AL', {'Produto': ['Óleo Combustível'], 'Total do volume': [45231]})
('AL', {'Produto': ['Óleo Diesel'], 'Total do volume': [7858]})
('AM', {'Produto': ['Etanol Hidratado'], 'Total do volume': [172

###Pipeline Diesel

In [8]:
# Tabela retornando produto diesel, por estado e volume total

ufs = [
 'AC',
 'AL',
 'AM',
 'AP',
 'BA',
 'CE',
 'DF',
 'ES',
 'GO',
 'MA',
 'MG',
 'MS',
 'MT',
 'PA',
 'PB',
 'PE',
 'PI',
 'PR',
 'RJ',
 'RN',
 'RO',
 'RR',
 'RS',
 'SC',
 'SE',
 'SP',
 'TO'
 ]
 
for j in range(len(ufs)):
  
  pij = beam.Pipeline()
  pipe1 = (
          pij
          |f'Extrair do CSV{i}{j}'>> beam.io.ReadFromText('/content/dataset_final_derivados.csv',skip_header_lines = 0)
          |f'Separador do CSV{i}{j}'>> beam.Map(lambda record: record.split(','))
          |f'Filtrar diesel{i}{j}'>> beam.Filter(lambda record: (record[2] == 'Óleo Diesel' and (record[1])==ufs[j]))
          |f'Agregar as Colunas{i}{j}'>> beam.Map(lambda record: (record[1],round(float(record[4]))))
          |f'Construir uma nova tabela{i}{j}'>> beam.CombinePerKey(sum)
          |f'1Construir uma nova tabela diesel{i}{j}'>> beam.Distinct()
          
      )

  pipe2 = (
          pij
          |f'{i}{j}Extrair do CSV'>> beam.io.ReadFromText('/content/dataset_final_derivados.csv',skip_header_lines = 0)
          |f'{i}{j}Separador do CSV'>> beam.Map(lambda record: record.split(','))
          |f'{i}{j}Filtrar derivado diesel'>> beam.Filter(lambda record: (record[2] == 'Óleo Diesel' and (record[1])==ufs[j]))
          |f'{i}{j}Agregar as Colunas'>> beam.Map(lambda record: (record[1],record[2]))
          |f'{i}{j}Construir uma nova tabela de derivado diesel'>> beam.Distinct()
          
      )

  tabela = (
      {'Produto':pipe2, 'Total do volume':pipe1 }
      |f'{i}{j}Agrupar as pernas' >> beam.CoGroupByKey()
      |f'{i}{j}Resultado'>> beam.Map(print)
      
  )


  pij.run()

('AC', {'Produto': ['Óleo Diesel'], 'Total do volume': [3726]})
('AL', {'Produto': ['Óleo Diesel'], 'Total do volume': [7858]})
('AM', {'Produto': ['Óleo Diesel'], 'Total do volume': [20496]})
('AP', {'Produto': ['Óleo Diesel'], 'Total do volume': [4992]})
('BA', {'Produto': ['Óleo Diesel'], 'Total do volume': [60402]})
('CE', {'Produto': ['Óleo Diesel'], 'Total do volume': [18889]})
('DF', {'Produto': ['Óleo Diesel'], 'Total do volume': [8620]})
('ES', {'Produto': ['Óleo Diesel'], 'Total do volume': [21832]})
('GO', {'Produto': ['Óleo Diesel'], 'Total do volume': [50493]})
('MA', {'Produto': ['Óleo Diesel'], 'Total do volume': [23316]})
('MG', {'Produto': ['Óleo Diesel'], 'Total do volume': [138480]})
('MS', {'Produto': ['Óleo Diesel'], 'Total do volume': [26768]})
('MT', {'Produto': ['Óleo Diesel'], 'Total do volume': [51543]})
('PA', {'Produto': ['Óleo Diesel'], 'Total do volume': [40984]})
('PB', {'Produto': ['Óleo Diesel'], 'Total do volume': [8923]})
('PE', {'Produto': ['Óleo Die